In [ ]:
import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling1D, Layer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
import keras_nlp
import warnings
warnings.filterwarnings('ignore')
import tqdm as notebook_tqdm

# Positional Encoding Layer
class PositionalEncoding(Layer):
    def __init__(self):
        super(PositionalEncoding, self).__init__()

    def call(self, x):
        seq_len = tf.shape(x)[1]
        d_model = tf.shape(x)[2]

        pos = tf.range(seq_len, dtype=tf.float32)[:, tf.newaxis]
        i = tf.range(d_model, dtype=tf.float32)[tf.newaxis, :]

        angle_rates = 1 / tf.pow(10000., (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        angle_rads = pos * angle_rates

        sines = tf.sin(angle_rads[:, 0::2])
        cosines = tf.cos(angle_rads[:, 1::2])

        pos_encoding = tf.concat([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[tf.newaxis, ...]

        return x + pos_encoding

# Load dữ liệu
with open('Datasets/train.json', 'r') as f:
    train_data = json.load(f)
with open('Datasets/test.json', 'r') as f:
    test_data = json.load(f)

train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)

# Chuẩn hóa embedding
train_embeddings = train_data['audio_embedding'].apply(np.array)
max_len = train_embeddings.apply(len).max()
embedding_dim = len(train_embeddings.iloc[0][0])

train_X = pad_sequences(train_embeddings, maxlen=max_len, dtype='float32', padding='post', truncating='post')
train_Y = train_data['is_turkey'].values

valid_idx = test_data['audio_embedding'].apply(lambda x: isinstance(x, list) and len(x) > 0)
test_embeddings = test_data.loc[valid_idx, 'audio_embedding'].apply(np.array)
test_X = pad_sequences(test_embeddings, maxlen=max_len, dtype='float32', padding='post', truncating='post')

# StandardScaler
B, T, D = train_X.shape
scaler = StandardScaler()
train_X = scaler.fit_transform(train_X.reshape(-1, D)).reshape(B, T, D)
test_X = scaler.transform(test_X.reshape(-1, D)).reshape(test_X.shape[0], T, D)

# Mô hình với Positional Encoding + TransformerEncoder
def build_transformer_with_positional_encoding(seq_len, d_model):
    inputs = Input(shape=(seq_len, d_model))
    x = PositionalEncoding()(inputs)

    for _ in range(3):
        x = keras_nlp.layers.TransformerEncoder(
            intermediate_dim=512,
            num_heads = 8 ,
            dropout=0.3,
        )(x)

    x = GlobalAveragePooling1D()(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation="relu")(x)
    x = Dropout(0.5)(x)
    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(inputs, outputs)
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["AUC"])
    return model

# Huấn luyện và chọn mô hình tốt
qualified_models = []
for i in range(200):
    print(f"\n--- Training iteration {i} ---")
    train_X_, val_X, train_Y_, val_Y = train_test_split(train_X, train_Y, test_size=0.3, random_state=45)

    model = build_transformer_with_positional_encoding(seq_len=max_len, d_model=embedding_dim)
    early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=0)

    model.fit(train_X_, train_Y_,
              validation_data=(val_X, val_Y),
              epochs=100,
              batch_size=32,
              callbacks=[early_stop],
              verbose=0)

    y_pred_prob = model.predict(val_X, verbose=0).ravel()
    y_pred = (y_pred_prob >= 0.5).astype(int)

    precision = precision_score(val_Y, y_pred)
    recall = recall_score(val_Y, y_pred)
    auc = roc_auc_score(val_Y, y_pred_prob)
    acc = accuracy_score(val_Y, y_pred)
    f1 = f1_score(val_Y, y_pred)

    if precision >= 0.94 and recall >= 0.94 and auc >= 0.99 and f1 >= 0.94 and acc >= 0.94:
        qualified_models.append({
            'model': model,
            'auc': auc,
            'accuracy': acc,
            'precision': precision,
            'recall': recall,
            'f1': f1
        })

# Sắp xếp mô hình
qualified_models = sorted(qualified_models, key=lambda x: -x['precision'])

# In kết quả
if qualified_models:
    print("\n=== Top models with precision >= 0.94 and recall >= 0.94 ===")
    for i, m in enumerate(qualified_models):
        print(f"[Model {i}]")
        print(f"  AUC       : {m['auc']:.4f}")
        print(f"  Accuracy  : {m['accuracy']:.4f}")
        print(f"  Precision : {m['precision']:.4f}")
        print(f"  Recall    : {m['recall']:.4f}")
        print(f"  F1 Score  : {m['f1']:.4f}")
else:
    print("Không có mô hình nào đạt các tiêu chí >= 0.94.")

# Dự đoán trên test set
if qualified_models:
    best_model = qualified_models[0]['model']
    test_pred_prob = best_model.predict(test_X, verbose=0).ravel()
    test_data['is_turkey'] = -1.0
    test_data.loc[valid_idx, 'is_turkey'] = test_pred_prob
    test_data.loc[valid_idx, ['vid_id', 'is_turkey']].to_csv('result.csv', index=False)
    print("\n✅ Đã lưu kết quả dự đoán từ mô hình tốt nhất vào 'result.csv'")


c:\Users\pc\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



--- Training iteration 0 ---


--- Training iteration 1 ---

--- Training iteration 2 ---

--- Training iteration 3 ---

--- Training iteration 4 ---

--- Training iteration 5 ---

--- Training iteration 6 ---

--- Training iteration 7 ---

--- Training iteration 8 ---

--- Training iteration 9 ---
